## Import

In [19]:
import tensorflow as tf
import tensorflow_hub as hub

import numpy as np
import pandas as pd
import os
import io
import random

# Split data
from sklearn.model_selection import train_test_split

## Functions

In [3]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
      y_true: true labels in the form of a 1D array
      y_pred: predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [4]:
import matplotlib.pyplot as plt

def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.

  Args:
    history: TensorFlow model History object (see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History)
  """ 
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend()

## Download the Kaggle NLP-Getting-Started Dataset

The text samples of Tweets labelled as disaster or not disaster

In [5]:
url = 'https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip'

zip_dir = tf.keras.utils.get_file(origin=url, extract=True)
zip_dir

'C:\\Users\\bruce\\.keras\\datasets\\nlp_getting_started.zip'

In [6]:
base_dir = os.path.dirname(zip_dir)
train_csv = os.path.join(base_dir, 'train.csv')
test_csv = os.path.join(base_dir, 'test.csv')

train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

for label, count in enumerate(train_df.target.value_counts()):
    print(f'There are {count} sentences labelled as {label} in Training dataset')

# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # frac=1 -> 100% of the data

# Split 80% Training data and 20% Validation data
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled['text'].to_numpy(),
    train_df_shuffled['target'].to_numpy(),
    train_size=0.8,
    random_state=42
)

print('---')
print('Number of Training data:', len(train_sentences))
print('Number of Validation data:', len(val_sentences))
print('Number of Testing data:', len(test_df))

There are 4342 sentences labelled as 0 in Training dataset
There are 3271 sentences labelled as 1 in Training dataset
---
Number of Training data: 6090
Number of Validation data: 1523
Number of Testing data: 3263


## Use 10% of the training data

In [7]:
# train_10_percent = train_df_shuffled[['text', 'target']].sample(frac=0.1, random_state=42)
# train_10_percent_sentences = train_10_percent['text']
# train_10_percent_labels = train_10_percent['target']

# split = int(0.1 * len(train_sentences))
# train_10_percent_sentences = train_sentences[:split]
# train_10_percent_labels = train_labels[:split]

## Tokenization/Vectorization

In [8]:
# Average number of tokens (words) in the training tweets
max_len = round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

max_vocab_len = 10000

text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_vocab_len, # bounded the number of most occurrences of words (auto add <OOV>)
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None, # groups of n-words
    output_mode='int', # how to map tokens to numbers
    output_sequence_length=max_len, # how long the sequences to be
    pad_to_max_tokens=True
)

# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

# Get all the unique words in our training data
words_in_vocab = text_vectorizer.get_vocabulary()
count = 5
print(f'The most common {count} words are: {words_in_vocab[:count]}')
print(f'The least common {count} words are: {words_in_vocab[-count:]}')

The most common 5 words are: [['', '[UNK]', 'the', 'a', 'in']]
The least common 5 words are: [['minded', 'mindblowing', 'milne', 'milledgeville', 'millcityio']]


## Download the pretrained model

In [9]:
url = 'https://storage.googleapis.com/ztm_tf_course/08_model_6_USE_feature_extractor.zip'

zip_dir = tf.keras.utils.get_file(origin=url, extract=True)
zip_dir

'C:\\Users\\bruce\\.keras\\datasets\\08_model_6_USE_feature_extractor.zip'

## Load the model

In [10]:
model_dir = os.path.join(os.path.dirname(zip_dir), '08_model_6_USE_feature_extractor')
model = tf.keras.models.load_model(model_dir)

## Evaluate the mode

In [11]:
with tf.device('/CPU:0'): # this line fix error
  eval = model.evaluate(val_sentences, val_labels, verbose=1)

for name, value in zip(model.metrics_names, eval):
  print("%s: %.10f" % (name, value))

48/48 [==============================] - 2s 25ms/step - loss: 0.4099 - accuracy: 0.8201
loss: 0.4099445343
accuracy: 0.8200919032


## Predictions

In [12]:
with tf.device('/CPU:0'): # this line fix error
    pred_probs = model.predict(val_sentences)
preds = tf.squeeze(tf.round(pred_probs))
preds[:10]

48/48 [==============================] - 1s 21ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [13]:
results = calculate_results(val_labels, preds)
results

{'accuracy': 82.00919238345371,
 'precision': 0.8211717138273148,
 'recall': 0.8200919238345371,
 'f1': 0.8183988036567054}

## Finding the most wrong predictions

### Create a DataFrame of different params for each of the test image

In [14]:
df = pd.DataFrame({
    'text': val_sentences,
    'target': val_labels,
    'pred': preds,
    'pred_probs': tf.squeeze(pred_probs)
})
df

,text,target,pred,pred_probs
0,DFR EP016 Monthly Meltdown - On Dnbheaven 2015...,0,0.0,0.159757
1,FedEx no longer to transport bioterror germs i...,0,1.0,0.747162
2,Gunmen kill four in El Salvador bus attack: Su...,1,1.0,0.988749
3,@camilacabello97 Internally and externally scr...,1,0.0,0.196229
4,Radiation emergency #preparedness starts with ...,1,1.0,0.707808
...,...,...,...,...
1518,@SidelineSavage what like a pipe made of peanu...,0,0.0,0.114509
1519,Avalanche City - Sunset http://t.co/48h3tLvLXr...,1,0.0,0.287812
1520,The Whirlwind! Scourge of Europe! RT @whedones...,0,0.0,0.249553
1521,ENGLAND EAST COAST. Dogger Bank Westward. 1. S...,0,0.0,0.432494


### Find the wrong predictions and sort by prediction probabilities

In [15]:
most_wrong = df[df['target'] != df['pred']].sort_values('pred_probs', ascending=False)
most_wrong

,text,target,pred,pred_probs
881,@adorableappple No reported flooding po in the...,0,1.0,0.970023
1307,We should all have a fire safety plan. RT @Mat...,0,1.0,0.939019
846,Mourning notices for stabbing arson victims st...,0,1.0,0.938105
1502,Two Jewish Terrorists Charged In Historic-Chur...,0,1.0,0.918810
31,? High Skies - Burning Buildings ? http://t.co...,0,1.0,0.910196
...,...,...,...,...
233,I get to smoke my shit in peace,1,0.0,0.042087
935,'I did another one I did another one. You stil...,1,0.0,0.041997
38,Why are you deluged with low self-image? Take ...,1,0.0,0.038998
244,Reddit Will Now QuarantineÛ_ http://t.co/pkUA...,1,0.0,0.038949


### False Positive predictions

True label is `0` but prediction is `1`

In [16]:
for row in most_wrong[:3].itertuples():
    _, text, target, pred, pred_probs = row
    print(f'Target: {target}, Pred: {pred}, Prob: {pred_probs}')
    print(f'Text:\n{text}\n')
    print('----\n')

Target: 0, Pred: 1.0, Prob: 0.9700231552124023
Text:
@adorableappple No reported flooding po in the area. Ten-4. #mmda

----

Target: 0, Pred: 1.0, Prob: 0.9390192627906799
Text:
We should all have a fire safety plan. RT @Matt_Kroschel: MOCK WILDFIRE near #Vail as agencies prepare for the worst. http://t.co/SWwyLRk0fv

----

Target: 0, Pred: 1.0, Prob: 0.9381048679351807
Text:
Mourning notices for stabbing arson victims stir Û÷politics of griefÛª in Israel: Posters for Shira Banki and A... http://t.co/3GZ5zQQTHe

----



### False Negative predictions

True label is `1` but prediction is `0`

In [17]:
for row in most_wrong[-3:].itertuples():
    _, text, target, pred, pred_probs = row
    print(f'Target: {target}, Pred: {pred}, Prob: {pred_probs}')
    print(f'Text:\n{text}\n')
    print('----\n')

Target: 1, Pred: 0.0, Prob: 0.038997940719127655
Text:
Why are you deluged with low self-image? Take the quiz: http://t.co/XsPqdOrIqj http://t.co/CQYvFR4UCy

----

Target: 1, Pred: 0.0, Prob: 0.03894944489002228
Text:
Reddit Will Now QuarantineÛ_ http://t.co/pkUAMXw6pm #onlinecommunities #reddit #amageddon #freespeech #Business http://t.co/PAWvNJ4sAP

----

Target: 1, Pred: 0.0, Prob: 0.03718578442931175
Text:
Ron &amp; Fez - Dave's High School Crush https://t.co/aN3W16c8F6 via @YouTube

----



## Making predictions on the Test dataset

### Pick predictions randomly

In [23]:
test_sentences = test_df['text'].to_list()
test_random_samples = random.sample(test_sentences, 3)
for text in test_random_samples:
    with tf.device('/CPU:0'): # this line fix error
        pred_prob = tf.squeeze(model.predict([text]))
    pred = tf.round(pred_probs)
    print(f'Pred: {pred} Prob: {pred_prob}')
    print(f'Text:\n{text}\n')
    print('----\n')

1/1 [==============================] - 0s 66ms/step
Pred: 0.0 Prob: 0.9805117249488831
Text:
wo Pic of 16yr old PKK suicide bomber who detonated bomb in Turkey Army trench released http://t.co/Sj57BoKsiB /'/'//

----

1/1 [==============================] - 0s 53ms/step
Pred: 0.0 Prob: 0.09325683861970901
Text:
@freeagent1717 @ChaseIngram @amandagiroux28 lol I saw something like that last cops was quarantined taking him out the house etc

----

1/1 [==============================] - 0s 53ms/step
Pred: 0.0 Prob: 0.16871552169322968
Text:
Sometimes blood ain't no thicker than water and sometimes family will bring you down quicker than strangers ???????

----

